In [ ]:
! export WANDB_PROJECT=lora-light

In [ ]:
!git clone https://github.com/lunaczp/alpaca-lora.git
%cd alpaca-lora

In [ ]:
!pip install -r requirements.txt

In [ ]:
!python finetune.py \
    --base_model 'NousResearch/Llama-2-7b-hf' \
    --data_path 'alpaca_data_gpt4.json' \
    --output_dir './lora-alpaca' \
    --batch_size 128 \
    --micro_batch_size 4 \
    --num_epochs 3 \
    --learning_rate 1e-4 \
    --cutoff_len 512 \
    --val_set_size 2000 \
    --lora_r 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --lora_target_modules '[q_proj,v_proj]' \
    --train_on_inputs \
    --group_by_length

In [ ]:
! pip install -U "huggingface_hub[cli]"

In [ ]:
! huggingface-cli login
! cd ./lora-alpaca && huggingface-cli upload LightXXXXX/llama-2-7b-hf-lora-alpaca-light . 

In [ ]:
#test
! python generate.py \
    --load_8bit \
    --base_model 'NousResearch/Llama-2-7b-hf' \
    --lora_weights 'LightXXXXX/llama-2-7b-hf-lora-alpaca-light'


In [ ]:
# merge
! export BASE_MODEL='NousResearch/Llama-2-7b-hf'
! export LORA_MODEL='LightXXXXX/llama-2-7b-hf-lora-alpaca-light'
! python export_hf_checkpoint.py

In [ ]:
#upload
! cd ./hf_ckpt && huggingface-cli upload LightXXXXX/llama-2-7b-hf-alpaca-light . .

In [ ]:
# convert to gguf
%cd /content/
! git clone https://github.com/ggerganov/llama.cpp
%cd llama.cpp
! pip install -r requirements.txt
%cd ../


In [ ]:
#build f16
! mkdir models
! python3 llama.cpp/convert.py --outtype f16 --outfile models/llama-2-7b-alpaca-light-f16.gguf hf_ckpt/

In [ ]:
# quantize
%cd llama.cpp
! make
! ./quantize ../models/llama-2-7b-alpaca-light-f16.gguf ../models/llama-2-7b-alpaca-light-Q4_K_M.gguf Q4_K_M
%cd ../

In [ ]:
# upload to huggingface
%cd models
! huggingface-cli upload LightXXXXX/llama-2-7b-alpaca-light-gguf . .